In [ ]:
import os
import datetime
import logging
import shutil
import pandas as pd
import glob
import csv
#from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
import threading
import os
import shutil
import time
import multiprocessing
import re
import os
import subprocess
import sys
import shutil
import glob
import logging
import csv
import os
import shutil
import logging
import numpy as np
import pandas as pd
import math


from FUNCTION import make_top_protein, fill_water_ions, energy_min, make_new_minim_nvt_npt, make_new_minim_config_samd, run_md
from FUNCTION import files_gmxmmpbsa, gmx_mmpbsa, Data_Analysis_Pre, Data_Analysis_Cal, clean_for_each_cycle,make_new_mutation, GRO_to_PDB
from FUNCTION import Data_Analysis_Cal_child

#PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
PROJECT_ROOT = os.getcwd()

DATA_DIR = os.path.join(PROJECT_ROOT, "DATA")
VMD_DIR = os.path.join(PROJECT_ROOT, "VMD_FUNCTION")
FUNCTION_DIR = os.path.join(PROJECT_ROOT, "FUNCTION")
FORCE_FIELD_PATH = os.path.join(PROJECT_ROOT, "FORCE_FIELD")
MMPBSA_INFILE_PATH = os.path.join(PROJECT_ROOT, "gmx_mmpbsa_in")
# pdb file
protein_infile = "HLA_BiAB_protein_50ns" 
#protein_infile = "mtbind"
protein_file_path = os.path.join(DATA_DIR, f"{protein_infile}.pdb")

# MDP files
ions_mdp_file = "ions"
minim_mdp_file = "minim"
nvt_mdp_file = "NVT"
npt_mdp_file = "NPT"
samd_mdp_file = "SAMD"
md_mdp_file = "EngComp_ff14sb_custom"
only_protein_md_mdp_file = "Protein_EngComp_ff14sb_custom"

ions_mdp_path = os.path.join(DATA_DIR, f"{ions_mdp_file}.mdp")
minim_mdp_path = os.path.join(DATA_DIR, f"{minim_mdp_file}.mdp")
nvt_mdp_path = os.path.join(DATA_DIR, f"{nvt_mdp_file}.mdp")
npt_mdp_path = os.path.join(DATA_DIR, f"{npt_mdp_file}.mdp")
samd_mdp_path = os.path.join(DATA_DIR, f"{samd_mdp_file}.mdp")
md_mdp_path = os.path.join(DATA_DIR, f"{md_mdp_file}.mdp")
only_protein_md_mdp_path = os.path.join(DATA_DIR, f"{only_protein_md_mdp_file}.mdp")



def create_output_directory():
    
    current_dir = os.getcwd()
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir_name = f"output_{timestamp}"
    output_dir_path = os.path.join(current_dir, output_dir_name)
    
    os.mkdir(output_dir_path)
    print(f"Created directory: {output_dir_path}")

    os.chdir(output_dir_path)

    
    
    return output_dir_path

ROOT_OUTPUT = create_output_directory()

logging.basicConfig(
    filename = "OUTPUT.out",
    level = logging.INFO,
    format="%(asctime)s - %(levelname)s -%(message)s"
)

logging.info(f"ROOT FOLDER PATH: {ROOT_OUTPUT}")

In [ ]:
def build_folders(current_dir, cycle_num):
    # Create folder for each cycle
    folders  ={}
    
    for cycle_n in range (1,cycle_num + 1):
        folder_name = f"cycle{cycle_n}_MD"
        folder_path = os.path.join(current_dir, folder_name)
        os.makedirs(folder_path, exist_ok = True)
        folders[f"cycle{cycle_n}_MD"] = folder_path

    folders["repository"] = os.path.join(current_dir,"REPOSITORY")
    folders["TEMP_FILES_FOLDER"] = os.path.join(current_dir,"TEMP_FILES_FOLDER")
    folders["REMOVED_FILES_FOLDER"] = os.path.join(current_dir,"REMOVED_FILES_FOLDER")
    folders["results"] = os.path.join(current_dir,"RESULTS")

    for folder in folders.values():
        os.makedirs(folder,exist_ok = True)
  
    header = [
    "#RUNnumber", "DeltaG(kJ/mol)", "Coul(kJ/mol)", "vdW(kJ/mol)",
    "PolSol(kJ/mol)", "NpoSol(kJ/mol)", "ScoreFunct", "ScoreFunct2",
    "Canonica_AVG", "MedianDG", "DeltaG_2s", "dG_PotEn"]

    df = pd.DataFrame(columns=header)
    results_file_path = os.path.join(folders["results"], "MoleculesResults.dat")
    df.to_csv(results_file_path, sep='\t', index=False, header=True)

    return folders
## OC2 DOESN'T NORMAL ONE, NEED TO CHANGE
def add_ter_to_pdb(pdb_file_name):
    
    temp_file_name = f"{pdb_file_name}_temp"  
    with open(pdb_file_name, 'r') as f:
        lines = f.readlines()

    new_lines = []  
    for i, line in enumerate(lines):
        new_lines.append(line)
        if "OC2" in line:
            if i + 1 >= len(lines) or not lines[i + 1].startswith("TER"):
                new_lines.append("TER\n")

    with open(temp_file_name, 'w') as f:
        f.writelines(new_lines)

    os.rename(temp_file_name, pdb_file_name)

def replace_his_residues_flexible(input_pdb, output_pdb):
    with open(input_pdb, "r", encoding="utf-8") as infile, open(output_pdb, "w", encoding="utf-8") as outfile:
        for line in infile:
            if line.startswith("ATOM"):
                match = re.match(r"(.{17})(HISE|HISD|HISP)(.*)", line)
                if match:
                    line = f"{match.group(1)}{'HIS':<4}{match.group(3)}"
            
            outfile.write(line.rstrip('\n') + '\n')

def MD_for_each_cycle(work_dir, cycle_number,input_structure_file,samd_mdp_path,output_gro, sequence, md_mdp_path, tpr_file, trj_name):
    print("start MD in MD function")
    #cycle_number = 1
    #while cycle_number <= cycle_num:
        
    cycle_MD_path = folders[f"cycle{cycle_number}_MD"] 
    os.chdir(work_dir)
    shutil.copy(os.path.join(folders["repository"], "system_equil.gro"), "./")
    shutil.copy(os.path.join(folders["repository"], "topol.top"), "./")

    for itp_file in glob.glob(os.path.join(folders["repository"], "*rotein_chain_*.itp")):
        shutil.copy(itp_file, "./")

    for itp_file in glob.glob(os.path.join(folders["repository"], "posres_*.itp")):
        shutil.copy(itp_file, "./")

    for cpt_file in glob.glob(os.path.join(folders["repository"], "*NPT*.cpt")):
        shutil.copy(cpt_file, "./")

    #make_new_minim_config_samd("system_equil.gro", samd_mdp_path, "system_Compl_MDstart", 0)
    make_new_minim_config_samd(input_structure_file, samd_mdp_path, output_gro, sequence)
    #run_md(md_mdp_path,"system_Compl_MD", "traj_MD", 0, 1)
    run_md(md_mdp_path, tpr_file, trj_name, sequence, cycle_number)
    shutil.copy("system_Compl_MD.gro", f"LastFrame_cycle{cycle_number}.gro")
    #cycle_number += 1


def gmx_mmpbsa_for_each_cycle(work_dir, cycle_number,only_protein_md_mdp_path,VMD_DIR,temp_files_folder, FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, REMOVED_FILES_FOLDER, results_folder, repository_folder, current_conf_path):
    #cycle_number = 1
    #while cycle_number <= cycle_num:
    ConfName = f"cycle{cycle_number}"
    RootName = f"cycle{cycle_number}_BE"
    cycle_number_MD_FOLDER = folders[f"cycle{cycle_number}_MD"]
    # 输出相关信息
    print(f"Cycle Number: {cycle_number}")
    print(f"Configuration Name: {ConfName}")
    print(f"Root Name: {RootName}")
    print(f"MD Folder Path: {cycle_number_MD_FOLDER}")
    #os.chdir(cycle_number_MD_FOLDER )
    os.chdir(work_dir)
        
    repository_pdb_file = os.path.join(repository_folder, f"{protein_infile}.pdb")
    #startingFrameGMXPBSA="2000"
    # make files for gmx_mmpbsa
    # files_gmxmmpbsa(starting_gro_file, repository_pdb_file, trj_file, tpr_file, top_file, mdp_name, root_name, conf_name, vmd_function_folder, temp_files_folder)

    files_gmxmmpbsa("system_Compl_MD", repository_pdb_file, "traj_MD", "system_Compl_MD", "topol", only_protein_md_mdp_path, RootName, ConfName, VMD_DIR, temp_files_folder, cycle_number)
    # get number of frames
    try:
        with open("trj_check.out", "r") as file:
            number_of_frames = next(
                (line.split()[1] for line in file if line.startswith("Step")), None
            )
    except FileNotFoundError:
        print(f"Error: File trj_check.out not found.")
        number_of_frames = None
    #conda_activate_path="/home/bio/ls/bin"
    conda_path = shutil.which("conda")
    conda_activate_path = os.path.dirname(conda_path)
    conda_gmxmmpbsa_name="gmxMMPBSA"
    forcefield="amber99sb-ildn"
    #FORCE_FIELD_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/FORCE_FIELD"
    mmpbsa_inFILE="mmpbsa_LinearPB_amber99SB_ILDN.in"
    #MMPBSA_INFILE_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/gmx_mmpbsa_in"
    np_value = 32
    #gmx_mmpbsa(1, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, 
    #             mmpbsa_inFILE, MMPBSA_INFILE_PATH , np_value, number_of_frames)
    gmx_mmpbsa(cycle_number, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, mmpbsa_inFILE, MMPBSA_INFILE_PATH, np_value, number_of_frames)
    # data analysis
    NUMframe = "all"
    Data_Analysis_Pre(cycle_number_MD_FOLDER, REMOVED_FILES_FOLDER, NUMframe)
    Data_Analysis_Cal(cycle_number, results_folder)
    # clean and move files
    clean_for_each_cycle(cycle_number, repository_folder, cycle_number_MD_FOLDER, RootName, REMOVED_FILES_FOLDER, current_conf_path)
        # 更新周期号及相关变量
    #cycle_number += 1
    #conf_name = f"cycle{cycle_number}"
    #root_name = f"cycle{cycle_number}_BE"
    #cycle_number_md_folder = os.path.join(current_conf_path, f"cycle{cycle_number}_MD")

def run_cycle(cycle_number, cycle_num, md_args, gmx_args):
    """
    处理当前 cycle 的 gmx_mmpbsa 和下一个 cycle 的 MD。
    """
    process = []

    # 启动当前 cycle 的 gmx_mmpbsa
    gmx_process = multiprocessing.Process(target=gmx_mmpbsa_for_each_cycle, args=(folders[f"cycle{cycle_number}_MD"],cycle_number, *gmx_args))
    process.append(gmx_process)
    gmx_process.start()

    # 如果还有下一个 cycle，启动下一个 cycle 的 MD
    if cycle_number < cycle_num:
        next_md_process = multiprocessing.Process(target=MD_for_each_cycle, args=(folders[f"cycle{cycle_number+1}_MD"], cycle_number + 1, *md_args))
        process.append(next_md_process)
        next_md_process.start()

    # 等待所有线程完成
    for p in process:
        p.join()

In [ ]:
max_mutant = 2
cycle_num = 2 # the run cycle numbers for each configuration
MUTANT_signal = False

for sequence in range (0,max_mutant+1):
    try:
        os.chdir(ROOT_OUTPUT)
    except OSError:
        logging.error(f"Cannot enter {ROOT_OUTPUT} folder")
        exit()

    if sequence == 0:
        
        # create configuration folder
        configuration_path = os.path.join(ROOT_OUTPUT,"configuration")
        os.mkdir(configuration_path)
        current_path_store = configuration_path
        
        print(f"Create directory: {configuration_path}")
        os.chdir(configuration_path)
        logging.info(f"#### Begin with configuration{sequence} ####")
        logging.info(f"PATH : {configuration}")
    else:
        mutant_folder_path = os.path.join(ROOT_OUTPUT,f"Mutant{sequence}")
        os.mkdir(mutant_folder_path)
        current_path_store = mutant_folder_path
        
        print(f"Create directort: {mutant_folder_path}")
        os.chdir(mutant_folder_path)
        logging.info(f"#### Begin with Mutant{sequence} ####")
        logging.info(f"PATH : {mutant_folder_path}")

    if MUTANT_signal == True:
        attempts = 1
        new_mutant = True
        while new_mutant == True:
            # pdb_file, res_position, chain, new_restype, res_pos_list,res_weight_files, new_restype_list, keep_hydration, output_name
            pdb_file = protein_infile #LastFRame_xxxx.pdb
            res_position = None
            chain = None
            new_restype = None
            res_pos_list = "28:H 29:H 30:H 31:H 32:H 52:H 54:H 55:H 56:H 57:H 98:H 100:H 102:H 105:H 93:L 94:L 95:L"
            res_weight_files = None
            new_restype_list = ['LEU', 'VAL', 'ILE', 'MET', 'PHE', 'TYR', 'TRP','GLU', 'ASP','ARG', 'LYS','SER', 'THR', 'ASN', 'GLN', 'HIS']
            output_name = f"Mutant{sequence}"
            if attemps < 6:
                logging.info("Making a new mutation (Keep hydration).")
                keep_hydration = True
                make_new_mutation(pdb_file, res_position, chain, new_restype, res_pos_list,res_weight_files, new_restype_list, keep_hydration, output_name)
            else:
                logging.info("Making a new mutation (Random).")
                keep_hydration = False
                make_new_mutation(pdb_file, res_position, chain, new_restype, res_pos_list,res_weight_files, new_restype_list, keep_hydration, output_name)
        attempts += 1
        new_mutant =False
        # Check that the sequence hasn't be tested already (self avoiding walk)
        os.remove(pdb_file)
        protein_infile= f"Mutant{sequence}"

        protein_file_path= os.path.join(current_path_store, f"{protein_infile}.pdb")
        destination_file = os.path.join(current_path_store, f"{protein_infile}_noH.pdb")
        shutil.copy(protein_file_path, destination_file)


    Metropolis_flag= 0
    current_dir = os.getcwd()
    folders = build_folders(current_dir,cycle_num)

    # generating a topology and build box
    make_top_protein(protein_file_path, "amber99sb-ildn", "tip3p", "system", "topol")

    # cp system.pdb {protein_infile}.pdb in current folder
    source = os.path.join(current_dir, "system.pdb")
    destination = os.path.join(current_dir, f"{protein_infile }.pdb")
    try:
        shutil.copy(source,destination)
    except Exception:
        print("Copy system.pdb failed.")

    add_ter_to_pdb(f"{protein_infile }.pdb")
    output_pdb = os.path.join(ROOT_OUTPUT, f"{protein_infile}.pdb")
    replace_his_residues_flexible(f"{protein_infile}.pdb",output_pdb)
    # Adding water and ions
    fill_water_ions("system", "topol", ions_mdp_path)
    # Energy Minimiization
    energy_min(minim_mdp_path, "system_ions", "topol", "system_compl")

    
    # Nvt and Npt
    make_new_minim_nvt_npt("system_compl_minim.gro", nvt_mdp_path, npt_mdp_path, "system_equil", 0)

    # Move .cpt, .top, and .itp files to repository folder
    for file_pattern in [f"{current_dir}/*.cpt", f"{current_dir}/*.top", f"{current_dir}/*.itp"]:
        for file in glob.glob(file_pattern):
            shutil.move(file, folders["repository"])

    # Move specific files to repository folder
    shutil.move(f"{current_dir}/{protein_infile}.pdb", folders["repository"])
    shutil.move(f"{current_dir}/system_compl_minim.gro", folders["repository"])
    shutil.move(f"{current_dir}/system_equil.gro", folders["repository"])


    # Move temp* and *out files to removed files folder
    for file in glob.glob("./*temp*.*") + glob.glob("./*.temp") + glob.glob("./*out"):
        shutil.move(file, folders["REMOVED_FILES_FOLDER"])

    # Remove files with # in their name
    for file in glob.glob("./#*"):
        os.remove(file)

    md_args = ("system_equil.gro",samd_mdp_path,"system_Compl_MDstart", sequence, md_mdp_path, "system_Compl_MD", "traj_MD")
    gmx_args = (only_protein_md_mdp_path,VMD_DIR,folders["TEMP_FILES_FOLDER"], FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, folders["REMOVED_FILES_FOLDER"], folders["results"], folders["repository"], configuration_path)
    # 1st cycle MD
    MD_for_each_cycle(folders["cycle1_MD"],1, *md_args)

    # each cycle: gmx_mmpbsa and next MD
    for cycle_number in range(1, cycle_num + 1):
        run_cycle(cycle_number, cycle_num, md_args, gmx_args)

    
    last_cycle_MD_FOLDER = os.path.join(folders["repository"],f"cycle{cycle_number}_MD")
    last_cycle_gro = os.path.join(last_cycle_MD_FOLDER,f"LastFrame_cycle{cycle_number}.gro")
    shutil.copy(last_cycle_gro, os.path.join(folders["repository"],f"LastFrame_cycle{cycle_number}.gro"))
    logging.info(f"Making the starting PDB for the next Mutation from LastFrame_cycle{cycle_number}.gro")
    
    os.chdir(current_path_store)
    
    repository_pdb_file = os.path.join(folders["repository"], f"{protein_infile}.pdb")

    pathGRO = folders["repository"]
    fileNameGRO = f"LastFrame_cycle{cycle_number}"
    pathPDB = os.path.dirname(repository_pdb_file)
    pdb_name_with_extension = os.path.basename(repository_pdb_file) #xxxx.pdb
    pdb_name_without_extension = os.path.splitext(pdb_name_with_extension)[0] #xxxx
    fileNamePDB = pdb_name_without_extension
    FileNamePDB_OUT = f"LastFrame_cycle{cycle_number}"
    GRO_to_PDB(pathGRO, fileNameGRO, pathPDB, fileNamePDB, FileNamePDB_OUT, VMD_DIR, folders["TEMP_FILES_FOLDER"])
    last_cycle_pdb = os.path.join(folders["repository"], f"LastFrame_cycle{cycle_number}.pdb")
    add_ter_to_pdb(last_cycle_pdb)        
    output_last_cycle_pdb = os.path.join(ROOT_OUTPUT, f"Mutant{sequence}_cycle{cycle_number}_LastFrameMD.pdb")
    replace_his_residues_flexible(last_cycle_pdb,output_last_cycle_pdb)
    # update protein_infile
    protein_infile = f"Mutant{sequence}_cycle{cycle_number}_LastFrameMD"
    logging.info("Making the average of the cycles results.")
    
    all_cycle_data = "All_cycle_data.out"
    MoleculesResults_data = os.path.join(folders["results"], "MoleculesResults.dat")
    data_analysis_temp = "DataAnalysis_temp.csv"

    if os.path.exists(all_cycle_data):
        os.remove(all_cycle_data)

    flag_header = True

    with open(MoleculesResults_data, 'r') as infile, open(all_cycle_data, 'w') as outfile:
        for line in infile:
            if flag_header == True:
                # 写入标题行，保留格式
                outfile.write(f"#{'configNum':<10} \t{line}")
                flag_header = False
            else:
                # 写入其他数据行
                outfile.write(f"{'avg':<10} \t{line}")
    # 使用 pandas 读取文件并提取第二列及以后的数据
    df = pd.read_csv(all_cycle_data, sep='\t')  # 读取数据，使用制表符分隔
    df_filtered = df.iloc[:, 1:]  # 提取第二列及后续列

    # 将结果保存为 CSV 格式
    df_filtered.to_csv(data_analysis_temp, sep = '\t', index=False,header = False)
    #Data_Analysis_Signal = False
    Data_Analysis_Cal_child(data_analysis_temp, "AllData.temp", False)
    '''
    # 格式化输出文件
    with open("AllData.temp", 'r') as infile, open("AllData.temp", 'w') as outfile:
        for line in infile:
            # 替换格式，添加额外的制表符
            line = line.replace("#AVG", "#AVG\t\t").replace("#STD", "#STD\t\t")
            outfile.write(line)
    '''
    frame_count = 0
    # 提取 #AVG 和 #STD 的行并追加到 AllData.out
    with open("AllData.temp", 'r') as temp_file, open(all_cycle_data, 'a') as outfile:
        for line in temp_file:

            if line.startswith("#frame"):
                frame_count +=1
                if frame_count ==2:
                    outfile.write(line)
            if line.startswith("#AVG") or line.startswith("#STD"):
                outfile.write(line)
    '''
    # remove temp file
    for temp_file in [all_data_temp, data_analysis_temp]:
        shutil.move(temp_file, os.path.join(removed_files_folder, os.path.basename(temp_file)))
    '''
    frame = []
    avg = []
    std = []
    with open(all_cycle_data, 'r') as infile:
    # 逐行解析文件内容
        for line in infile:
            if line.startswith("#frame"):
                frame = line.strip().split()[1:]
            elif line.startswith("#AVG"):
                avg = line.strip().split()[1:]
            elif line.startswith("#STD"):
                std = line.strip().split()[1:]
    if frame and avg and std:
        output_lines = ["Results for Configuation"]
        output_lines += [f"{frame[i]}: {avg[i]} +- {std[i]} kJ/mol"
                        for i in range(len(frame))
                       ]
        logging.info("\n".join(output_lines))
    shutil.move("AllData.temp", folders["REMOVED_FILES_FOLDER"])
    shutil.move(data_analysis_temp, folders["REMOVED_FILES_FOLDER"])
    